# HEP Jet assignment project - Data analysis and particle finding script

## Import essential packages.
---
* We will use [uproot](https://github.com/scikit-hep/uproot) packages to parse our .root file.
* The content of function `particle properties` and `jet properties` is defined in `particle_properties.py` and `jet_properties.py`.

In [7]:
import uproot
import pandas as pd 
import numpy as np 
from particle_properties_uproot import particle_properties  #import particle properties helper function from particle_properties.py
from jet_properties_uproot import jet_properties  #import jet properties helper function from jet_properties.py
import h5py

## Loading data, determine parameters, and assign variable
---

In [8]:
data  = uproot.open('./tag_1_delphes_events.root')['Delphes']
#data.show()

particle = particle_properties(data)
jet = jet_properties(data)

Length = len(particle.event)
test_length = 10

PID_W_plus = 24 
PID_W_minus = -24
PID_DOWN = 1
PID_DOWN_VAR = -1
PID_UP = 2
PID_UP_BAR = -2
PID_STRANGE = 3
PID_STRANGE_BAR = -3
PID_CHARM = 4
PID_CHARM_BAR = -4
PID_BOTTOM = 5
PID_BOTTOM_BAR = -5
PID_TOP = 6
PID_TOP_BAR = -6

top_idx = np.zeros(len(particle.event))
top_daughter_idx_1 = np.zeros(len(particle.event))
top_daughter_pid_1 = np.zeros(len(particle.event))
top_daughter_idx_2 = np.zeros(len(particle.event))
top_daughter_pid_2 = np.zeros(len(particle.event))

top_bar_idx = np.zeros(len(particle.event))
top_bar_daughter_idx_1 = np.zeros(len(particle.event))
top_bar_daughter_pid_1 = np.zeros(len(particle.event))
top_bar_daughter_idx_2 = np.zeros(len(particle.event))
top_bar_daughter_pid_2 = np.zeros(len(particle.event))

# W_plus_idx = np.zeros(len(particle.event))
# W_minus_idx = np.zeros(len(particle.event))

# b_quark = np.zeros(len(particle.event))
# b_bar_quark = np.zeros(len(particle.event))

# quark_idx_1 = np.zeros(len(particle.event))
# quark_idx_2 = np.zeros(len(particle.event))
# quark_idx_3 = np.zeros(len(particle.event))
# quark_idx_4 = np.zeros(len(particle.event))

patron_array = np.zeros([ len(particle.event) , 6, 6])

In [9]:
# frame = np.zeros((1, 60, 80))
# with h5py.File('mytestfile.hdf5','w') as f:
#     dset = f.create_dataset('video', data=frame, maxshape=(None, 60, 80), chunks=True)
# with h5py.File("mytestfile.hdf5", "r") as f:
#     print(f.keys())

In [10]:
patron_array.shape

(10000, 6, 6)

## Event selection 
---
1. Must contain:
    * At least 2 b tagged jet.
    * At least 6 jet exists.
    * For each jet, require |$\eta$| < 2.4 and $P_{T}$ > 20GeV. 

In [11]:
#Generate maker for each stage(event selection and jet selection.)
marker_event = []
marker_jet = []

for i in range(len(jet.event)):
    marker_event.append(0)
    marker_jet.append(np.zeros([len(jet.pt[i])]))


marker_event = np.asanyarray(marker_event)
marker_jet = np.asanyarray(marker_jet)

print(type(marker_event), type(marker_jet))
print(marker_event.shape, marker_jet.shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(10000,) (10000,)
/usr/local/lib64/python3.6/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [12]:
#Mark which event pass the selection
print("+-----------------------------------------------------------------------------------------------------+")
print("Start event selection.")
for i in range(len(jet.event)):
    min_pt = np.min(jet.pt[i])
    num_of_eta_in_range = np.sum(jet.eta[i] < 2.4 ) 
    num_of_jet = len(jet.pt[i])
    num_of_btagged = np.sum(jet.btag[i] == 1)
    if min_pt > 20 and num_of_eta_in_range >= 6 and num_of_jet >=6 and num_of_btagged >= 2: 
        marker_event[i] = 1
    else :
        pass
print("Event selection doen.")
print("+-----------------------------------------------------------------------------------------------------+")

#Mark which jet in each event pass the selection.
print("+-----------------------------------------------------------------------------------------------------+")
print("Start jet selection.")
for i in range(len(jet.event)):
    for j in range(len(jet.pt[i])):
        if marker_event[i] == 1:
            if jet.btag[i][j] == 1 and jet.pt[i][j] > 20 and jet.eta[i][j] <= 2.4:
                marker_jet[i] == 1
                marker_jet[i][j] == 1 
            else :
                pass
        else :
            pass 
print("Jet selection doen.")
print("+-----------------------------------------------------------------------------------------------------+")

+-----------------------------------------------------------------------------------------------------+
Start event selection.
Event selection doen.
+-----------------------------------------------------------------------------------------------------+
+-----------------------------------------------------------------------------------------------------+
Start jet selection.
Jet selection doen.
+-----------------------------------------------------------------------------------------------------+


In [13]:
df0 = particle.dataframelize(8)
#df1 = particle.dataframelize(1)
#df2 = particle.dataframelize(2)

## Particle tracing and daughter finding section
---

In [14]:
def shift_particle_tracing(dataset, PID_d, idx):
    if (dataset.iloc[idx,6] == PID_d):
        return dataset.iloc[idx,4]

def particle_tracing(dataset, PID, STATUS):

    for i in range(len(dataset)):
        if(dataset.iloc[i,1] == STATUS and dataset.iloc[i,6] == PID ): 
            daughter_index = int(dataset.iloc[i,0])
    if( dataset.iloc[daughter_index,6] == PID ):
        shifted_particle_index = dataset.iloc[daughter_index, 4]


    while dataset.iloc[shifted_particle_index,6] == PID:
            init_shifted_particle_index = shifted_particle_index
            shifted_particle_index = shift_particle_tracing(dataset, PID, init_shifted_particle_index)       

    dauthter_idx_1 = dataset.iloc[init_shifted_particle_index, 4]
    daughter_pid_1 = dataset.iloc[dauthter_idx_1, 6]

    dauthter_idx_2 = dataset.iloc[init_shifted_particle_index, 5]
    daughter_pid_2 = dataset.iloc[dauthter_idx_2, 6]

    return init_shifted_particle_index, dauthter_idx_1, daughter_pid_1, dauthter_idx_2, daughter_pid_2


In [15]:
for i in range(0,10):
    print("+------------------------------------------------------------------------------------------------------+")
    print("Start parsing event : {0}\nStart to trace top quark and find its daughters.".format(i))
    top_idx[i], top_daughter_idx_1[i], top_daughter_pid_1[i], top_daughter_idx_2[i], top_daughter_pid_2[i] = particle_tracing(particle.dataframelize(i), PID_TOP, 22)
    print("+------------------------------------------------------~-----------------------------------------------+")
    print("Start to find top_bar quark and its daughters.")
    top_bar_idx[i], top_bar_daughter_idx_1[i], top_bar_daughter_pid_1[i], top_bar_daughter_idx_2[i], top_bar_daughter_pid_2[i] = particle_tracing(particle.dataframelize(i), PID_TOP_BAR, 22)
    print("+------------------------------------------------------------------------------------------------------+")

+------------------------------------------------------------------------------------------------------+
Start parsing event : 0
Start to trace top quark and find its daughters.
+------------------------------------------------------~-----------------------------------------------+
Start to find top_bar quark and its daughters.
+------------------------------------------------------------------------------------------------------+
+------------------------------------------------------------------------------------------------------+
Start parsing event : 1
Start to trace top quark and find its daughters.
+------------------------------------------------------~-----------------------------------------------+
Start to find top_bar quark and its daughters.
+------------------------------------------------------------------------------------------------------+
+------------------------------------------------------------------------------------------------------+
Start parsing event : 2
S

In [ ]:
### Tracing the daughter 

In [22]:
#Input two daughter of top/top_bar and find their daughter
def quark_finder(dataset, mother_idx_1, mother_idx_2):
    
    #Specific two daughter of top
    def W_b_specifier(dataset, input_1_idx, input_2_idx):
        if dataset.iloc[int(input_1_idx),6] == PID_W_plus or dataset.iloc[int(input_1_idx),6] == PID_W_minus :
            return int(input_1_idx), int(dataset.iloc[int(input_1_idx),6]), int(input_2_idx)
        elif dataset.iloc[int(input_1_idx),6] == PID_BOTTOM or dataset.iloc[int(input_1_idx),6] == PID_BOTTOM_BAR :
            return  int(input_2_idx), int(dataset.iloc[int(input_1_idx),6]), int(input_1_idx)
        else :
            print("Please check your data.")
    
    W_boson_idx, mother_pid, b_quark_idx = W_b_specifier(dataset, mother_idx_1, mother_idx_2)
    
    #Find the two daughters of boson
    
    daughter_1_idx = dataset.iloc[W_boson_idx, 4]
    daughter_1_pid = dataset.iloc[daughter_1_idx, 6]
    daughter_2_idx = dataset.iloc[W_boson_idx, 5]
    daughter_2_pid = dataset.iloc[daughter_2_idx, 6]

    
    if daughter_1_pid == mother_pid or daughter_2_pid == mother_pid:

        init_idx = W_boson_idx
        daughter_pid = daughter_1_pid
        if daughter_2_pid == mother_pid:
            daughter_pid = daughter_2_pid
        while daughter_pid == mother_pid :
            daughter_1_idx = dataset.iloc[int(init_idx), 4]
            daughter_2_idx = dataset.iloc[int(init_idx), 5]

            daughter_1_pid = dataset.iloc[int(daughter_1_idx), 6]
            daughter_2_pid = dataset.iloc[int(daughter_2_idx), 6]

            daughter_pid = daughter_1_pid
            init_idx = daughter_1_idx
            if daughter_2_pid == mother_pid:
                daughter_pid = daughter_2_pid
                init_idx = daughter_2_idx
            
            
            print("Temporary daughter 1 indxe: {0}, PID: {1}".format(daughter_1_idx, daughter_1_pid))
            print("Temporary daughter 2 indxe: {0}, PID: {1}".format(daughter_2_idx, daughter_2_pid))
        # init_idx = W_boson_idx
        # while daughter_2_pid == mother_pid:
        #     daughter_2_idx = dataset.iloc[int(init_idx), 5]
        #     daughter_2_pid = dataset.iloc[int(daughter_2_idx), 6]
        #     init_idx = daughter_2_idx
        #     print("Temporary daughter 2 indxe: {0}, PID: {1}".format(daughter_2_idx, daughter_2_pid))
    
    print("Found daughter 1 index: {0}, PID: {1}.\nFound daughter 2 index: {2}, PID: {3}".format(daughter_1_idx, daughter_1_pid, daughter_2_idx, daughter_2_pid))
    return  b_quark_idx, daughter_1_idx, daughter_2_idx

In [23]:
for i in range(0,10):
    if marker_event[i] == 1 :
        print("+------------------------------------------------------------------------------------------------------+")
        print("Start parsing event : {0}\nStart to find top quark's daughters.".format(i))
        patron_array[i][0][0], patron_array[i][1][0], patron_array[i][2][0] = quark_finder(particle.dataframelize(i), top_daughter_idx_1[i], top_daughter_idx_2[i])
        print("+------------------------------------------------------~-----------------------------------------------+")
        print("Start to find top_bar quark's daughters.")
        patron_array[i][3][0], patron_array[i][4][0], patron_array[i][5][0], = quark_finder(particle.dataframelize(i), top_bar_daughter_idx_1[i], top_bar_daughter_idx_2[i])
        print("+------------------------------------------------------------------------------------------------------+")
    elif marker_event[i] == 0 :
        patron_array[i] = 'Nan'
    else: pass

+------------------------------------------------------------------------------------------------------+
Start parsing event : 8
Start to find top quark's daughters.
Temporary daughter 1 indxe: 407, PID: 24
Temporary daughter 2 indxe: 407, PID: 24
Temporary daughter 1 indxe: 446, PID: 4
Temporary daughter 2 indxe: 447, PID: -3
Found daughter 1 index: 446, PID: 4.
Found daughter 2 index: 447, PID: -3
+------------------------------------------------------~-----------------------------------------------+
Start to find top_bar quark's daughters.
Temporary daughter 1 indxe: 360, PID: -24
Temporary daughter 2 indxe: 360, PID: -24
Temporary daughter 1 indxe: 400, PID: -24
Temporary daughter 2 indxe: 401, PID: 22
Temporary daughter 1 indxe: 414, PID: 1
Temporary daughter 2 indxe: 415, PID: -2
Found daughter 1 index: 414, PID: 1.
Found daughter 2 index: 415, PID: -2
+------------------------------------------------------------------------------------------------------+


In [18]:
for i in range(0,10):
    if marker_event[i] == 1:
        for j in range(0,6):
            dataset = particle.dataframelize(i)
            patron_array[i][j][1] = dataset.iloc[int(patron_array[i][j][0]), 6]  #PDGID
            patron_array[i][j][2] = dataset.iloc[int(patron_array[i][j][0]), 7]  #Pt
            patron_array[i][j][3] = dataset.iloc[int(patron_array[i][j][0]), 8]  #Eta
            patron_array[i][j][4] = dataset.iloc[int(patron_array[i][j][0]), 9]  #Phi
            patron_array[i][j][5] = dataset.iloc[int(patron_array[i][j][0]), 10]  #Mass

In [13]:
# for i in range(0,10):
#     print(patron_array[i])

## Patron-jet matching section
---

### Define the function for computing delta_R


In [14]:
def deltaPhi(phi1,phi2):
    phi = phi1-phi2
    while phi >= np.pi: phi -= np.pi*2.
    while phi < -np.pi: phi += np.pi*2.
    return phi

def delta_R(eta1, phi1, eta2, phi2):
    return np.sqrt(deltaPhi(phi1,phi2)**2+(eta1-eta2)**2)

def min_delta_R(target_1, target_2):
    pass

In [15]:
dR_between_patron_jet = []
dR_between_patron_patron = []

for i in range(len(jet.event)):
    dR_between_patron_jet.append(np.zeros([len(jet.pt[i]) * 6])) # # of connection = num of jet * num of patron
    dR_between_patron_patron.append(np.zeros([15])) # C^{6}_{2} = 15

dR_between_patron_jet = np.asanyarray(dR_between_patron_jet)
dR_between_patron_patron = np.asanyarray(dR_between_patron_patron)



In [49]:
max_num_of_jet_cand = []
for i in range(len(jet.event)):
    max_num_of_jet_cand.append(len(jet.pt[i]))
max_num_of_jet_cand = np.asanyarray(max_num_of_jet_cand)
max_num_of_jet = max_num_of_jet_cand.max()
print(max_num_of_jet)

patron_jet_matching = np.zeros([len(jet.event), 6, 2])

14


In [16]:
# for i in range(0,10):
#     print(dR_between_patron_jet[i].shape)

(36,)
(18,)
(18,)
(30,)
(36,)
(36,)
(42,)
(24,)
(42,)
(42,)


### Computing delta_R between each patron and jet

In [17]:
for i in range(test_length):
    if marker_event[i] == 1:
        j = 0
        a = 0
        b = 0
        while a < 6 :
            for b in range( len(jet.pt[i]) ):
                print(i, a, b)
                print(delta_R( patron_array[i][a][3], patron_array[i][a][4], jet.eta[i][b], jet.phi[i][b]))
                dR_between_patron_jet[i][j] = delta_R( patron_array[i][a][3], patron_array[i][a][4], jet.eta[i][b], jet.phi[i][b])
                j +=1
            a += 1 
    else :
        dR_between_patron_jet[i] = 'Nan'
        
        

8 0 0
4.165814946462408
8 0 1
1.2409081541734457
8 0 2
1.4370510145964808
8 0 3
2.1470968766152545
8 0 4
2.393077086271537
8 0 5
2.1879551259629246
8 0 6
0.0561078375283632
8 1 0
2.6256207464651395
8 1 1
2.782948879409113
8 1 2
2.0264794615009776
8 1 3
0.09987564369420499
8 1 4
3.198124238125701
8 1 5
1.2434733652084748
8 1 6
2.2458551402310434
8 2 0
2.2180904523490477
8 2 1
2.093791106499452
8 2 2
1.1726524584815639
8 2 3
1.119654032421636
8 2 4
2.1354500845397135
8 2 5
0.06924021520448215
8 2 6
2.114348183694153
8 3 0
3.0131561260149633
8 3 1
0.04618709568092776
8 3 2
0.9781131015131724
8 3 3
2.7257403402641316
8 3 4
1.2511927750734346
8 3 5
2.156586918524361
8 3 6
1.1983259775432709
8 4 0
3.290976975662634
8 4 1
0.8996986696106917
8 4 2
0.04477846088242515
8 4 3
1.9338889257771585
8 4 4
1.2595089340050385
8 4 5
1.2126670425520003
8 4 6
1.348026118591255
8 5 0
3.2780148321302374
8 5 1
0.5791963771106374
8 5 2
0.43553563738356726
8 5 3
2.3509708614724603
8 5 4
0.9558522199866376
8 5 5

### Matching jet and patron by finding the Min(dR(patron, jet))

In [68]:
for i in range(test_length):
    if marker_event[i] == 1:
        print("+------------------------------------------------------------------------------------------------------+")
        # print(dR_between_patron_jet.shape)
        array = np.reshape(dR_between_patron_jet[i], [6, len(jet.pt[i])])
        print(array.shape)
        
        dataset = pd.DataFrame({'1': array[0,:], 
                                '2': array[1,:],
                                '3': array[2,:],
                                '4': array[3,:],
                                '5': array[4,:],
                                '6': array[5,:],
                                })
        for j in range(0,6):

            min_val = dataset.stack().min()
            if min_val < 0.4:
                print("Min val: {0}".format(min_val))
                min_idx, min_col = dataset.stack().idxmin()
                patron_jet_matching[i][j][0] = int(min_idx)
                patron_jet_matching[i][j][1] = int(min_col)
                print(min_idx, min_col)
                dataset = dataset.drop([min_col], axis=1)
                dataset = dataset.drop([min_idx], axis=0)
                print(dataset)
            else:
                pass 
        

+------------------------------------------------------------------------------------------------------+
(6, 7)
Min val: 0.04477846088242515
2 5
          1         2         3         4         6
0  4.165815  2.625621  2.218090  3.013156  3.278015
1  1.240908  2.782949  2.093791  0.046187  0.579196
3  2.147097  0.099876  1.119654  2.725740  2.350971
4  2.393077  3.198124  2.135450  1.251193  0.955852
5  2.187955  1.243473  0.069240  2.156587  1.602949
6  0.056108  2.245855  2.114348  1.198326  1.416436
Min val: 0.04618709568092776
1 4
          1         2         3         6
0  4.165815  2.625621  2.218090  3.278015
3  2.147097  0.099876  1.119654  2.350971
4  2.393077  3.198124  2.135450  0.955852
5  2.187955  1.243473  0.069240  1.602949
6  0.056108  2.245855  2.114348  1.416436
Min val: 0.0561078375283632
6 1
          2         3         6
0  2.625621  2.218090  3.278015
3  0.099876  1.119654  2.350971
4  3.198124  2.135450  0.955852
5  1.243473  0.069240  1.602949
Min val: 0.069

In [77]:
patron_index = np.zeros([len(jet.event), 6])
jet_index = np.zeros([len(jet.event), 6])

for i in range(len(jet.event)):
    if marker_event[i] == 1:
        for j in range(0,6):
            patron_index[i][j] = patron_jet_matching[i][j][0]
            jet_index[i][j] = patron_jet_matching[i][j][1]

In [78]:
patron_index

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

## Saved selected events
---

In [ ]:
#Save the event which pass the selection
with h5py.File("event_record.h5",'r') as f:
    group_jet = f.create_group('jet')
    group_jet['Parton_Index'] = 
    group_jet['Barcode'] = 
    group_jet['Pt'] = new_jet_pt
    group_jet['Eta'] = new_jet_eta
    group_jet['Phi'] = new_jet_phi
    group_jet['Mass'] = new_jet_mass
    group_jet['BTag'] = new_jet_Btag
    
    group_patron = f.create_group('parton')
    group_patron['Jet_Index'] = new_jet_idx
    group_patron['Barcode'] = 
    group_patron['Pt'] = new_jet_pt
    group_patron['Eta'] = new_jet_eta
    group_patron['Phi'] = new_jet_phi
    group_patron['Mass'] = new_jet_mass
    